In [104]:
import rasterio
import numpy as np
import pandas as pd
import glob
import copy

In [55]:
coefficients = pd.read_csv("./coefficients/dark-spot-coefficients_pd.csv")

In [83]:
coefficients

,filename,index,blue,green,red,nir,X,Y,blue_coefficients,green_coefficients,red_coefficients,nir_coefficients
0,20190302_105726_SV1-01_50cm_RD_11bit_RGBI,26280,479,442,331,401,4.408141,52.20792,-1,1,-16,-49
1,20190302_105829_SV1-01_50cm_RD_11bit_RGBI,26280,502,461,357,396,4.408141,52.20792,-24,-18,-42,-44
2,20190308_111644_SV1-01_50cm_RD_11bit_RGBI,26280,453,416,327,395,4.408141,52.20792,25,27,-12,-43
3,20190422_111335_SV1-01_50cm_RD_11bit_RGBI,27667,455,444,388,537,4.408141,52.20792,23,-1,-73,-185
4,20190601_105844_SV1-04_50cm_RD_11bit_RGBI,26280,695,643,467,643,4.408141,52.20792,-217,-200,-152,-291
5,20190721_112303_SV1-04_50cm_RD_11bit_RGBI,26280,670,616,445,558,4.408141,52.20792,-192,-173,-130,-206
6,20191025_113826_SV1-02_50cm_RD_11bit_RGBI,26280,322,249,155,87,4.408141,52.20792,156,194,160,265
7,20191130_110721_SV1-01_50cm_RD_11bit_RGBI,27667,98,66,89,88,4.408141,52.20792,380,377,226,264
8,20191202_110525_SV1-04_50cm_RD_11bit_RGBI,26280,412,370,289,364,4.408141,52.20792,66,73,26,-12
9,20200304_114601_SV1-02_SV_RD_11bit_RGBI,26280,432,394,263,305,4.408141,52.20792,46,49,52,47


In [114]:
for file in glob.glob("E:/data/coepelduynen/*ndvi_height.tif"):
    
    path_to_tif = file.replace("\\","/")
    print(path_to_tif)
    with rasterio.open(path_to_tif) as src:
        meta = src.meta
        data_input = src.read()
        
        try:

            data_input_copy = copy.deepcopy(data_input)
            data_input_1 = copy.deepcopy(data_input_copy[0])+coefficients[coefficients['filename'] == path_to_tif.split("/")[-1].split("_50cm")[0]]['blue_coefficients'].values[0]
            data_input_1[data_input_1 < 0] = 0
            data_input_2 = copy.deepcopy(data_input_copy[1])+coefficients[coefficients['filename'] == path_to_tif.split("/")[-1].split("_50cm")[0]]['green_coefficients'].values[0]
            data_input_2[data_input_2 < 0] = 0
            data_input_3 = copy.deepcopy(data_input_copy[2])+coefficients[coefficients['filename'] == path_to_tif.split("/")[-1].split("_50cm")[0]]['red_coefficients'].values[0]
            data_input_3[data_input_3 < 0] = 0
            data_input_4 = copy.deepcopy(data_input_copy[3])+coefficients[coefficients['filename'] == path_to_tif.split("/")[-1].split("_50cm")[0]]['nir_coefficients'].values[0]
            data_input_4[data_input_4 < 0] = 0

            data_output = np.array([data_input_1, data_input_2, data_input_3, data_input_4,copy.deepcopy( data_input_copy[4]), copy.deepcopy(data_input_copy[5])])

            del data_input
            del data_input_copy


            print(np.min(data_input_1))
            print(np.max(data_input_1))

            print(np.min(data_input_2))
            print(np.max(data_input_2))

            print(np.min(data_input_3))
            print(np.max(data_input_3))
            print(path_to_tif.replace(".tif","_normalized.tif") )
            with rasterio.open(path_to_tif.replace(".tif","_normalized.tif"), 'w', **meta) as outds:        
                        outds.write_band(1,data_output[0])
                        outds.write_band(2,data_output[1])
                        outds.write_band(3,data_output[2])
                        outds.write_band(4,data_output[3])
                        outds.write_band(5,data_output[4])
                        outds.write_band(6,data_output[5])
        except Exception as e:
            print(e)
            print("Meh")
    src.close()


E:/data/coepelduynen/20200304_114601_SV1-02_SV_RD_11bit_RGBI_50cm_Oegstgeest_natura2000_coepelduynen_cropped_ndvi_height.tif
46
1147
49
1313
52
1082
E:/data/coepelduynen/20200304_114601_SV1-02_SV_RD_11bit_RGBI_50cm_Oegstgeest_natura2000_coepelduynen_cropped_ndvi_height_normalized.tif
E:/data/coepelduynen/20200323_112115_SV1-03_SV_RD_11bit_RGBI_50cm_Oegstgeest_natura2000_coepelduynen_cropped_ndvi_height.tif
166
1645
192
1694
111
1541
E:/data/coepelduynen/20200323_112115_SV1-03_SV_RD_11bit_RGBI_50cm_Oegstgeest_natura2000_coepelduynen_cropped_ndvi_height_normalized.tif
E:/data/coepelduynen/20200326_114323_SV1-02_SV_RD_11bit_RGBI_50cm_Oegstgeest_natura2000_coepelduynen_cropped_ndvi_height.tif
index 0 is out of bounds for axis 0 with size 0
Meh
E:/data/coepelduynen/20200508_110812_SV1-01_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height.tif
12
862
0
932
0
1022
E:/data/coepelduynen/20200508_110812_SV1-01_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_croppe

In [54]:
path_to_tif = "E:/data/coepelduynen/20210815_111051_SV1-03_SV_RD_11bit_RGBI_50cm_Oegstgeest_natura2000_coepelduynen_cropped_ndvi_height.tif"
print(path_to_tif)
src = rasterio.open(path_to_tif)
meta = src.meta
data_input = src.read()
    

data_input[0] = data_input[0]+coefficients[coefficients['filename'] == path_to_tif.split("/")[-1].split("_50cm")[0]]['blue_coefficients'].values[0]
data_input[1] = data_input[1]+coefficients[coefficients['filename'] == path_to_tif.split("/")[-1].split("_50cm")[0]]['green_coefficients'].values[0]
data_input[2] = data_input[2]+coefficients[coefficients['filename'] == path_to_tif.split("/")[-1].split("_50cm")[0]]['red_coefficients'].values[0]
data_input[3] = data_input[3]+coefficients[coefficients['filename'] == path_to_tif.split("/")[-1].split("_50cm")[0]]['nir_coefficients'].values[0]

E:/data/coepelduynen/20210815_111051_SV1-03_SV_RD_11bit_RGBI_50cm_Oegstgeest_natura2000_coepelduynen_cropped_ndvi_height.tif


In [55]:
data_input

array([[[65476, 65476, 65476, ..., 65476, 65476, 65476],
        [65476, 65476, 65476, ..., 65476, 65476, 65476],
        [65476, 65476, 65476, ..., 65476, 65476, 65476],
        ...,
        [65476, 65476, 65476, ..., 65476, 65476, 65476],
        [65476, 65476, 65476, ..., 65476, 65476, 65476],
        [65476, 65476, 65476, ..., 65476, 65476, 65476]],

       [[65510, 65510, 65510, ..., 65510, 65510, 65510],
        [65510, 65510, 65510, ..., 65510, 65510, 65510],
        [65510, 65510, 65510, ..., 65510, 65510, 65510],
        ...,
        [65510, 65510, 65510, ..., 65510, 65510, 65510],
        [65510, 65510, 65510, ..., 65510, 65510, 65510],
        [65510, 65510, 65510, ..., 65510, 65510, 65510]],

       [[65506, 65506, 65506, ..., 65506, 65506, 65506],
        [65506, 65506, 65506, ..., 65506, 65506, 65506],
        [65506, 65506, 65506, ..., 65506, 65506, 65506],
        ...,
        [65506, 65506, 65506, ..., 65506, 65506, 65506],
        [65506, 65506, 65506, ..., 65506, 655

In [110]:
path_to_tif = "E:/data/coepelduynen/20200508_110812_SV1-01_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height.tif"
print(path_to_tif)
src = rasterio.open(path_to_tif)
meta = src.meta
data_input = src.read()

E:/data/coepelduynen/20200508_110812_SV1-01_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height.tif


In [111]:
data_input

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
   

In [40]:
np.max(data_input_1)

1439

In [76]:
data_input[0] = np.zeros(6398,4819)

TypeError: Cannot interpret '4819' as a data type

In [53]:
data_input[0]+coefficients[coefficients['filename'] == path_to_tif.split("/")[-1].split("_50cm")[0]]['blue_coefficients'].values[0]

array([[-60, -60, -60, ..., -60, -60, -60],
       [-60, -60, -60, ..., -60, -60, -60],
       [-60, -60, -60, ..., -60, -60, -60],
       ...,
       [-60, -60, -60, ..., -60, -60, -60],
       [-60, -60, -60, ..., -60, -60, -60],
       [-60, -60, -60, ..., -60, -60, -60]], dtype=int32)

In [56]:
coefficients[coefficients['filename'] == path_to_tif.split("/")[-1].split("_50cm")[0]]['blue_coefficients'].values[0]

-60

In [67]:
data_input[1]  = 5

In [69]:
data_input[1].shape

(6398, 4819)

In [99]:

data_input_1 = data_input[0]+coefficients[coefficients['filename'] == path_to_tif.split("/")[-1].split("_50cm")[0]]['blue_coefficients'].values[0]
data_input_2 = data_input[1]+coefficients[coefficients['filename'] == path_to_tif.split("/")[-1].split("_50cm")[0]]['green_coefficients'].values[0]
data_input_3 = data_input[2]+coefficients[coefficients['filename'] == path_to_tif.split("/")[-1].split("_50cm")[0]]['red_coefficients'].values[0]
data_input_4 = data_input[3]+coefficients[coefficients['filename'] == path_to_tif.split("/")[-1].split("_50cm")[0]]['nir_coefficients'].values[0]

data_input = np.array([data_input_1, data_input_2, data_input_3, data_input_4, data_input[4], data_input[5]])


In [100]:
data_input

array([[[ -60,  -60,  -60, ...,  -60,  -60,  -60],
        [ -60,  -60,  -60, ...,  -60,  -60,  -60],
        [ -60,  -60,  -60, ...,  -60,  -60,  -60],
        ...,
        [ -60,  -60,  -60, ...,  -60,  -60,  -60],
        [ -60,  -60,  -60, ...,  -60,  -60,  -60],
        [ -60,  -60,  -60, ...,  -60,  -60,  -60]],

       [[ -26,  -26,  -26, ...,  -26,  -26,  -26],
        [ -26,  -26,  -26, ...,  -26,  -26,  -26],
        [ -26,  -26,  -26, ...,  -26,  -26,  -26],
        ...,
        [ -26,  -26,  -26, ...,  -26,  -26,  -26],
        [ -26,  -26,  -26, ...,  -26,  -26,  -26],
        [ -26,  -26,  -26, ...,  -26,  -26,  -26]],

       [[ -30,  -30,  -30, ...,  -30,  -30,  -30],
        [ -30,  -30,  -30, ...,  -30,  -30,  -30],
        [ -30,  -30,  -30, ...,  -30,  -30,  -30],
        ...,
        [ -30,  -30,  -30, ...,  -30,  -30,  -30],
        [ -30,  -30,  -30, ...,  -30,  -30,  -30],
        [ -30,  -30,  -30, ...,  -30,  -30,  -30]],

       [[-128, -128, -128, ..., -128,

In [25]:
data_input